## 2021 금융데이터 경진대회 : 개미는 뚠뚠뚠 팀
### [[1. 웹크롤링 및 데이터 전처리 ]]
##### 목차
##### 1. 테마주 크롤링
##### 2. 뉴스기사 크롤링
##### 3. 기업재무정보 크롤링 및 재무지수 만들기

#### 1. 테마주 크롤링

#### 1) 라이브러리 불러오기

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os

In [3]:
## 코드 제출 파일경로로 바꿔주세요
os.chdir("C:/Users/User/Desktop/분석코드_개미는뚠뚠뚠")

#### 2) 웹 크롤링

In [4]:
theme_N = [] #### 테마를 담을 리스트입니다.
theme_link = [] #### 테마의 링크를 담을 링크입니다.

for i in range(1, 8): #### 페이지를 돌아가면서 반복합니다.
    url = 'https://finance.naver.com/sise/theme.nhn?&page='+ str(i)
    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    theme = soup.find_all("td", class_="col_type1") #### 테마의 목록을 선택합니다.
    for j in range(0, len(theme)):
        Nm = theme[j].get_text()
        link = theme[j].a['href']
        theme_N.append(Nm)
        theme_link.append(link)

In [5]:
theme_df =  pd.DataFrame({'theme' : theme_N, 'link' : theme_link}) #### 위에서 크롤링한 데이터를 데이터프레임으로 만듭니다.

In [6]:
theme_list = [] #### 테마를 담을 리스트입니다.
stock_n_list = [] #### 테마에 해당하는 회사를 담을 리스트입니다.
stock_expln_list = [] #### 테마 설명을 담을 리스트입니다.
for i in range(0, theme_df.shape[0]):
    url = 'https://finance.naver.com/' + theme_df.iloc[i,1] #### 페이지를 돌아가면서 반복합니다.
    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    theme_area = soup.find_all('div', class_='info_layer_wrap')
    for j in range(0, len(theme_area)):
        stock_n = theme_area[j].strong.get_text()
        stock_expln = theme_area[j].p.get_text()
        theme_list.append(theme_df.iloc[i,0])
        stock_n_list.append(stock_n)
        stock_expln_list.append(stock_expln)

In [7]:
theme_stock_df =  pd.DataFrame({'theme' : theme_list, 'stock' : stock_n_list, 'stock_expln' : stock_expln_list})

In [8]:
theme_stock_df.tail(5)

,theme,stock,stock_expln
5919,제4이동통신,릭스솔루션,퀀텀모바일 컨소시엄에 참여해 미래창조과학부 주관 2015년도 기간통신사업(주파수할당...
5920,제4이동통신,세종텔레콤,세종모바일 컨소시엄에 참여해 미래창조과학부 주관 2015년도 기간통신사업(주파수할당...
5921,제4이동통신,쏠리드,퀀텀모바일 컨소시엄에 참여해 미래창조과학부 주관 2015년도 기간통신사업(주파수할당...
5922,제4이동통신,CS,퀀텀모바일 컨소시엄에 참여해 미래창조과학부 주관 2015년도 기간통신사업 허가신청(...
5923,제4이동통신,콤텍시스템,퀀텀모바일 컨소시엄에 참여해 미래창조과학부 주관 2015년도 기간통신사업에 참여한 ...


In [9]:
##테마주 설명 제거
theme_stock_df = theme_stock_df[theme_stock_df['theme']!=theme_stock_df['stock']] #### 테마에 대한 설명이 한 행으로 들어갔지 때문에 제거해줍니다.
theme_stock_df = theme_stock_df.reset_index(drop = True)

#### 3) 데이터 전처리

#### (1) 종목 기본정보 불러오기

In [11]:
#### 종목 데이터 불러오기
stock_data = pd.read_csv("data/krx전종목기본정보.csv", encoding='CP949')

In [12]:
stock_data.shape ##상장회사 : 2577개

(2579, 12)

#### (2) stock_data 전처리

In [13]:
stock_cd_list = stock_data[['단축코드', '한글 종목명', '한글 종목약명']].drop_duplicates()

In [14]:
stock_cd_list.columns = ['stock_cd', 'stock_kor', 'stock']

In [16]:
theme_data = pd.merge(theme_stock_df, stock_cd_list, how='left', on='stock')

In [18]:
theme_data = theme_data[['theme', 'stock', 'stock_cd']]

In [19]:
theme_data.columns = ['theme', 'stock', '단축코드']

#### (3) 데이터 저장

In [21]:
theme_data.to_csv("data/테마주 및 업종명.csv", index=False, encoding="utf-8-sig")

#### 2. 뉴스기사 크롤링 

#### (1) 패키지 불러오기

In [22]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from konlpy.tag import Okt
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import os
import datetime

In [25]:
## 코드 제출 파일경로로 바꿔주세요
os.chdir("C:/Users/User/Desktop/분석코드_개미는뚠뚠뚠")

#### (2) 뉴스 크롤링

#### (1) 뉴스 목록의 링크 크롤링

In [26]:
href_list = [] #### 링크를 담을 리스트를 만듭니다.
date_list = [] #### 뉴스 발행 날짜를 담을 리스트를 만듭니다.

for i in range(0, 40):
    url = 'https://www.mk.co.kr/news/stock/business-information/?page=' + str(i) #### 페이지 수를 돌아가며 반복합니다.
    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    a = soup.select('#container_left > div.list_area > dl > dt > a') #### 뉴스 기사의 목록을 선택합니다.
    for j in range(0, len(a)):
        href = a[j]['href']
        href_list.append(href)
    b = soup.select('#container_left > div.list_area > dl > dd.desc > span.date') #### 뉴스기사의 날짜를 선택합니다.
    for k in range(0, len(b)):
        date = b[k].get_text()
        date_list.append(date)

In [27]:
df = pd.DataFrame({'date':date_list, 'href':href_list}) #### 크롤링한 결과를 데이터프레임으로 만듭니다.

In [28]:
title_list = [] #### 뉴스기사의 제목 리스트를 만듭니다.
txt_list = [] #### 뉴스기사의 내용을 담을 리스트를 만듭니다.
for i in range(0, df.shape[0]):
    try : 
        url = df.href[i]
        response = requests.get(url)
        html = response.text
        soup = BeautifulSoup(response.content.decode('euc-kr', 'replace'), "html.parser")
        txt = soup.select('#article_body > div')[0].get_text() #### 뉴스기사의 본문을 선택합니다.
        title = soup.select('#top_header > div > div > h1')[0].get_text() ### 뉴스기사의 제목을 선택합니다.
    
        title_list.append(title)
        txt_list.append(txt)
    except :
        title_list.append('error')
        txt_list.append('error')

In [29]:
df['title'] = title_list
df['txt'] = txt_list

In [30]:
df['date'] = [i[:10] for i in df['date']]
df['date'] = pd.to_datetime(df['date'])

In [31]:
df.head()

,date,href,title,txt
0,2021-09-05,https://www.mk.co.kr/news/stock/view/2021/09/8...,"IMM-LG화학, 배터리&ESG; 펀드 5천억 규모로 확대해 결성 초읽기",IMM크레딧솔루션(ICS)이 LG화학과 공동으로 조성하는 KBE(Korea Batt...
1,2021-09-05,https://www.mk.co.kr/news/stock/view/2021/09/8...,저평가 매력 부각되는 제일기획,국내 광고주가 올 하반기 기저효과 소진과 실적 둔화 우려 등으로 조정을 받으면서 밸...
2,2021-09-05,https://www.mk.co.kr/news/stock/view/2021/09/8...,"""외국인 이제 삼성전자 산다, 오를 일만 남았다"" 5백만 개미주주 웃을까",\n\n\n[이충우 기자]\n 올 한해 내내 지겹도록 삼성전자를 팔아오던 외국인 투...
3,2021-09-03,https://www.mk.co.kr/news/stock/view/2021/09/8...,"씨티케이, 세계적 직판 회사 ACN과 할랄 화장품 개발 업무 협약",\n\n\n 씨티케이는 세계적인 직접판매 회사 ACN과 동남아시아 시장을 타깃으로 ...
4,2021-09-03,https://www.mk.co.kr/news/stock/view/2021/09/8...,"코스닥 상장사 한스바이오메드, 사모펀드 투자받았다",인체 조직 이식재를 개발해 온 한스바이오메드가 사모펀드(PEF)의 투자를 받았다.\...


In [32]:
df.to_csv("data/뉴스기사 크롤링.csv", index=False, encoding="utf-8-sig")

#### 3. 기업재무정보 크롤링 및 재무지수 만들기

#### 1) 패키지 불러오기

In [34]:
import os
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from tqdm import tqdm
import numpy as np
import time
from sklearn.preprocessing import StandardScaler

In [35]:
## 코드 제출 파일경로로 바꿔주세요
os.chdir("C:/Users/User/Desktop/분석코드_개미는뚠뚠뚠")

#### 2) 상장 회사(kospi, kosdaq) 데이터 전처리

In [36]:
sector_kospi = pd.read_csv("data/sector_kospi.csv", encoding = 'CP949') #### KRX 정보데이터시스템 세부안내 업종별분류현황 kosdaq (조회일자 : 20200331)
sector_kosdaq = pd.read_csv("data/sector_kosdaq.csv", encoding = 'CP949')#### KRX 정보데이터시스템 세부안내 업종별분류현황 kospi (조회일자 : 20200331)

In [37]:
sector_kospi = sector_kospi[['종목코드', '종목명', '업종명']] #### 필요한 열만 선택합니다.
sector_kosdaq = sector_kosdaq[['종목코드', '종목명', '업종명']]

In [38]:
stock_data = pd.concat([sector_kospi, sector_kosdaq])
stock_data = stock_data.reset_index(drop = True)

In [39]:
stocks = stock_data['종목코드'] 
stocks = [i.zfill(6) for i in stocks] #### 종목코드는 6자리이기에 0을채워 6자리로 맞추어 줍니다.
stock_data['종목코드'] = stocks

#### 3) 크롤링 함수 만들기

In [40]:
chromedriver = "chromedriver.exe"

In [41]:
def Naver_fs(company):
    try : #### 에러처리를 할 수 있게 만듭니다.
        url = 'https://navercomp.wisereport.co.kr/v2/company/c1040001.aspx?cmp_cd={}&menuType=block'.format(company) #### 변수를 입력받아 URL를 만듭니다.
        driver = webdriver.Chrome(chromedriver)
        driver.get(url)
    
        profit = driver.find_element_by_css_selector('#val_tab1') #### 수익성 페이지 버튼 변수를 만듭니다.
        growth = driver.find_element_by_css_selector('#val_tab2') #### 성장성 페이지 버튼 변수를 만듭니다.
        stability = driver.find_element_by_css_selector('#val_tab3') #### 안정성 페이지 버튼 변수를 만듭니다.
        activity = driver.find_element_by_css_selector('#val_tab4') #### 활동성 페이지 버튼 변수를 만듭니다.
    
        ids = [profit, growth, stability, activity] #### 페이지 버튼을 하나의 리스트로 만듭니다.
        sectors = ['수익성','성장성','안정성','활동성']
        df_all = []
        driver.find_element_by_css_selector('#frqTyp1').click() #### 분기버튼을 클릭합니다.
    
        for id, sector in zip(ids, sectors):
            id.click() #### 페이지 버튼을 반복문으로 차례대로 클릭합니다.
            time.sleep(0.5)
            driver.find_element_by_css_selector('#hfinGubun').click() #### 조회버튼을 클릭하빈다.
            html = driver.page_source
            soup = BeautifulSoup(html,'html.parser') #### 페이지에 있는 글을 html로 가져옵니다.
            table = soup.select('table') #### table을 가져옵니다.
            table_html = str(table)
            table_df_list = pd.read_html(table_html)
            df = table_df_list[6]
            df = df.iloc[:,:7]
            df.columns = ['항목','2020/03','2020/06','2020/09','2020/12','2021/03','전분기대비'] #### 표의 열을 달아줍니다.
            df['투자지표'] = sector
            df = df.replace({'항목':'펼치기'}, {'항목':''},regex=True)
            df_all.append(df)
    
        first_term = soup.select('table > thead > tr > th > div')[1].get_text() #### 첫번째 재무정보의 분기를 가져옵니다.
        if first_term[:7] == '2020/03':
            term = '2003' #### 첫번째 재무정보의 분기가 20년 1분기인 경우 term을 2003으로 반환합니다.
        elif first_term[:7] == '2020/06':
            term = '2006'#### 첫번째 재무정보의 분기가 20년 2분기인 경우 term을 2006으로 반환합니다.
        else :
            term = 'NA' #### 위의 2가지 경우 모두 아닌경우 term을 NA로 처리해줍니다.
        
        
        indicators = pd.concat(df_all)
        indicators = indicators.set_index(['투자지표','항목'])
    
        driver.close()
    except :
        indicators = pd.DataFrame()
        term = 'NA' #### 에러가 났을 경우 NA로 처리해줍니다.

    return indicators, term

#### 4) 데이터 크롤링

In [42]:
stock_list = stock_data['종목코드'].tolist() #### 종목코드를 리스트로 만들어줍니다. 이후 크롤링함수에 반복문을 사용하여 넣을 예정입니다.

In [43]:
def error_na(dat, sector, column_nm): #### 에러 처리 함수를 만들어줍니다. 원하는 항목이 없을 경우 NA로 된 리스트를 반환합니다.
    try:
        df = dat.xs(sector, axis = 0).loc[column_nm].tolist()
        return(df)
    except:
        df = [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]
        return(df)

In [44]:
#### 재무정보 컬럼을 만들어줍니다. 반복문을사용하여 분기별로 10개의 재무정보 컬럼을 만들어줍니다.
#### 크롤링할 재무항목은 0000을 참고하여 선정하였습니다.
year_list = ['2003', '2006', '2009', '2012', '2103','2106']
for j in range(0, 6):
    globals()['순이익율_{}'.format(year_list[j])] = []
    globals()['ROE_{}'.format(year_list[j])] = []
    globals()['총자산증가율_{}'.format(year_list[j])] = []
    globals()['매출액증가율_{}'.format(year_list[j])] = []
    globals()['유동자산증가율_{}'.format(year_list[j])] = []
    globals()['유동비율_{}'.format(year_list[j])] = []
    globals()['부채비율_{}'.format(year_list[j])] = []
    globals()['총자산회전율_{}'.format(year_list[j])] = []
    globals()['매출채권회전율_{}'.format(year_list[j])] = []
    globals()['재고자산회전율_{}'.format(year_list[j])] = []

In [16]:
for i in tqdm(range(0, len(stock_list))): #### 상장회사 개수만큼 반복하여줍니다.
    stock_cd = stocks[i] #### 리스트에서 종목코드를 가져옵니다.
    result, term = Naver_fs(stock_cd) #### 위에서 만든 크롤링함수를 사용에 재무정보를 크롤링합니다.
    
    ##수익성 : 수익성과 관련된 항목만 뽑아 저장합니다. 만약 원하는 재무정보 항목이 없을 경우 에러처리 함수로 해당항목을 NA리스트로 만들어줍니다. 
    profit1 = error_na(result, '수익성', '  순이익률')
    profit2 = error_na(result, '수익성', '  ROE')
    
    ##성장성 : 성장성과 관련된 항목만 뽑아 저장합니다.
    growth1 = error_na(result, '성장성', '  총자산증가율')
    growth2 =error_na(result, '성장성', '  매출액증가율')
    growth3 =error_na(result, '성장성', '  유동자산증가율')
    
    
    ##안정성 : 안정성과 관련된 항목만 뽑아 저장합니다.
    stability1 = error_na(result, '안정성', '  유동비율')
    stability2 = error_na(result, '안정성', '  부채비율')
    
    ##활동성 : 활동성과 관련된 항목만 뽑아 저장합니다.
    activity1 = error_na(result, '활동성', '  총자산회전율')
    activity2 = error_na(result, '활동성', '  매출채권회전율')
    activity3 = error_na(result, '활동성', '  재고자산회전율')
    
    #### 첫분기 재무정보를 활용하여 각 회사의 재무정보의 시점을 맞추어 줍니다. 이를 하지 않으면 시점이 맞지 않게되어 왜곡된 정보를 사용할 수 있습니다.
    if term == '2006': #### 만약 첫 분기 재무정보가 20년 2분기인 경우 항복 제일 앞에 NA를 넣어줍니다.
        profit1.insert(0, np.nan)
        profit2.insert(0, np.nan)
        
        growth1.insert(0, np.nan)
        growth2.insert(0, np.nan)
        growth3.insert(0, np.nan)
        
        stability1.insert(0, np.nan)
        stability2.insert(0, np.nan)
        
        activity1.insert(0, np.nan)
        activity2.insert(0, np.nan)
        activity3.insert(0, np.nan)
        
    if term == '2003': #### 만약 첫 분기 재무정보가 20년 1분기인 경우 마지막에 NA를 넣어줍니다.
        profit1.insert(5, np.nan)
        profit2.insert(5, np.nan)
        
        growth1.insert(5, np.nan)
        growth2.insert(5, np.nan)
        growth3.insert(5, np.nan)
        
        stability1.insert(5, np.nan)
        stability2.insert(5, np.nan)
        
        activity1.insert(5, np.nan)
        activity2.insert(5, np.nan)
        activity3.insert(5, np.nan)
    
    if term == 'NA': #### 첫분기 항목이 둘다 아닌 경우 재무정보 리스트를 NA가 채워진 리스트로 반환합니다.
        profit1 = [np.nan] * 6
        profit2 = [np.nan] * 6
        
        growth1 = [np.nan] * 6
        growth2 = [np.nan] * 6
        growth3 = [np.nan] * 6
        
        stability1 = [np.nan] * 6
        stability2 = [np.nan] * 6
        
        activity1 = [np.nan] * 6
        activity2 = [np.nan] * 6
        activity3 = [np.nan] * 6
    
    
    #### 크롤링한 재무정보를 하나의 리스트에 넣습니다.
    for j in range(0, 6):
        years = year_list[j] #### 반복문을 사용하여 위에서 만든 컬럼에 차례대로 넣어줍니다.
        globals()['순이익율_' + years].append(profit1[j])
        globals()['ROE_' + years].append(profit2[j])
        
        globals()['총자산증가율_' + years].append(growth1[j])
        globals()['매출액증가율_' + years].append(growth2[j])
        globals()['유동자산증가율_' + years].append(growth3[j])
        
        globals()['유동비율_' + years].append(stability1[j])
        globals()['부채비율_' + years].append(stability2[j])
        
        globals()['총자산회전율_' + years].append(activity1[j])
        globals()['매출채권회전율_' + years].append(activity2[j])
        globals()['재고자산회전율_' + years].append(activity3[j])

100%|████████████████████████████████████████████████████████████████████████████| 2321/2321 [5:07:05<00:00,  7.94s/it]


In [52]:
fn_df = pd.DataFrame({'종목코드' : stock_data['종목코드'], #### 재무정보를 하나의 데이터프레임으로 만들어줍니다.
                    '종목명' : stock_data['종목명'], 
                    '업종명' : stock_data['업종명'], 
                      
                     '순이익율_2106' : 순이익율_2106,
                     'ROE_2106' : ROE_2106,
                     '총자산증가율_2106' : 총자산증가율_2106, 
                     '매출액증가율_2106' : 매출액증가율_2106, 
                     '유동자산증가율_2106' : 유동자산증가율_2106, 
                     '유동비율_2106' : 유동비율_2106, 
                     '부채비율_2106' : 부채비율_2106, 
                     '총자산회전율_2106' : 총자산회전율_2106, 
                     '매출채권회전율_2106' : 매출채권회전율_2106, 
                     '재고자산회전율_2106' : 재고자산회전율_2106,
                     
                     '순이익율_2103' : 순이익율_2103,
                     'ROE_2103' : ROE_2103,
                     '총자산증가율_2103' : 총자산증가율_2103, 
                     '매출액증가율_2103' : 매출액증가율_2103, 
                     '유동자산증가율_2103' : 유동자산증가율_2103, 
                     '유동비율_2103' : 유동비율_2103, 
                     '부채비율_2103' : 부채비율_2103, 
                     '총자산회전율_2103' : 총자산회전율_2103, 
                     '매출채권회전율_2103' : 매출채권회전율_2103, 
                     '재고자산회전율_2103' : 재고자산회전율_2103,
                     
                     '순이익율_2003' : 순이익율_2003,
                     'ROE_2003' : ROE_2003,
                     '총자산증가율_2003' : 총자산증가율_2003, 
                     '매출액증가율_2003' : 매출액증가율_2003, 
                     '유동자산증가율_2003' : 유동자산증가율_2003, 
                     '유동비율_2003' : 유동비율_2003, 
                     '부채비율_2003' : 부채비율_2003, 
                     '총자산회전율_2003' : 총자산회전율_2003, 
                     '매출채권회전율_2003' : 매출채권회전율_2003, 
                     '재고자산회전율_2003' : 재고자산회전율_2003,
                     
                     '순이익율_2006' : 순이익율_2006,
                     'ROE_2006' : ROE_2006,
                     '총자산증가율_2006' : 총자산증가율_2006, 
                     '매출액증가율_2006' : 매출액증가율_2006, 
                     '유동자산증가율_2006' : 유동자산증가율_2006, 
                     '유동비율_2006' : 유동비율_2006, 
                     '부채비율_2006' : 부채비율_2006, 
                     '총자산회전율_2006' : 총자산회전율_2006, 
                     '매출채권회전율_2006' : 매출채권회전율_2006, 
                     '재고자산회전율_2006' : 재고자산회전율_2006,
                     
                     '순이익율_2009' : 순이익율_2009,
                     'ROE_2009' : ROE_2009,
                     '총자산증가율_2009' : 총자산증가율_2009, 
                     '매출액증가율_2009' : 매출액증가율_2009, 
                     '유동자산증가율_2009' : 유동자산증가율_2009, 
                     '유동비율_2009' : 유동비율_2009, 
                     '부채비율_2009' : 부채비율_2009, 
                     '총자산회전율_2009' : 총자산회전율_2003, 
                     '매출채권회전율_2009' : 매출채권회전율_2009, 
                     '재고자산회전율_2009' : 재고자산회전율_2009,
                     
                     '순이익율_2012' : 순이익율_2012,
                     'ROE_2012' : ROE_2012,
                     '총자산증가율_2012' : 총자산증가율_2012, 
                     '매출액증가율_2012' : 매출액증가율_2012, 
                     '유동자산증가율_2012' : 유동자산증가율_2012, 
                     '유동비율_2012' : 유동비율_2012, 
                     '부채비율_2012' : 부채비율_2012, 
                     '총자산회전율_2012' : 총자산회전율_2012, 
                     '매출채권회전율_2012' : 매출채권회전율_2012, 
                     '재고자산회전율_2012' : 재고자산회전율_2012 })

In [53]:
fn_df.head()

,종목코드,종목명,업종명,순이익율_2106,ROE_2106,총자산증가율_2106,매출액증가율_2106,유동자산증가율_2106,유동비율_2106,부채비율_2106,...,순이익율_2012,ROE_2012,총자산증가율_2012,매출액증가율_2012,유동자산증가율_2012,유동비율_2012,부채비율_2012,총자산회전율_2012,매출채권회전율_2012,재고자산회전율_2012
0,095570,AJ네트웍스,서비스업,5.55,3.62,-13.73,2.05,-34.14,53.57,349.83,...,-0.89,-1.30,-8.10,-71.86,-18.52,50.03,451.43,0.05,0.96,1.61
1,006840,AK홀딩스,기타금융,-2.56,-12.84,2.58,22.92,1.17,68.24,273.04,...,-3.98,-20.57,0.79,-0.16,1.72,73.54,233.90,0.16,2.31,2.65
2,027410,BGF,기타금융,8.38,3.41,0.70,35.09,-0.49,843.42,7.83,...,-5.47,1.81,-0.26,-38.80,56.11,671.00,7.88,0.02,5.03,13.80
3,282330,BGF리테일,유통업,2.75,20.97,5.71,13.27,6.48,79.14,256.55,...,1.68,18.60,-3.12,-7.51,-9.29,82.63,240.28,0.65,57.27,10.75
4,138930,BNK금융지주,기타금융,21.77,7.66,4.65,9.03,NaN,NaN,1160.14,...,4.75,6.11,2.85,55.11,NaN,NaN,1085.89,NaN,NaN,NaN


In [19]:
fn_df.shape

(2321, 63)

In [21]:
fn_df.to_csv("data/재무정보크롤링.csv", index=False, encoding="utf-8-sig")

#### 5) 재무지수 만들기

#### (1) 데이터 전처리

In [54]:
finance_data = pd.read_csv("data/재무정보크롤링.csv") #### 위에서 크롤링한 데이터를 불러옵니다.

In [55]:
sector_kospi = pd.read_csv("data/sector_kospi.csv", encoding = 'CP949')
sector_kosdaq = pd.read_csv("data/sector_kosdaq.csv", encoding = 'CP949')

In [56]:
marcket_dat = pd.concat([sector_kospi[['종목코드', '시장구분']], sector_kosdaq[['종목코드', '시장구분']]])
marcket_dat['종목코드'] = [i.zfill(6) for i in marcket_dat['종목코드']] #### 종목코드가 6자리이므로 0을채워 자릿수를 맞추어줍니다.

In [57]:
finance_data.columns

Index(['종목코드', '종목명', '업종명', '순이익율_2106', 'ROE_2106', '총자산증가율_2106',
       '매출액증가율_2106', '유동자산증가율_2106', '유동비율_2106', '부채비율_2106', '총자산회전율_2106',
       '매출채권회전율_2106', '재고자산회전율_2106', '순이익율_2103', 'ROE_2103', '총자산증가율_2103',
       '매출액증가율_2103', '유동자산증가율_2103', '유동비율_2103', '부채비율_2103', '총자산회전율_2103',
       '매출채권회전율_2103', '재고자산회전율_2103', '순이익율_2003', 'ROE_2003', '총자산증가율_2003',
       '매출액증가율_2003', '유동자산증가율_2003', '유동비율_2003', '부채비율_2003', '총자산회전율_2003',
       '매출채권회전율_2003', '재고자산회전율_2003', '순이익율_2006', 'ROE_2006', '총자산증가율_2006',
       '매출액증가율_2006', '유동자산증가율_2006', '유동비율_2006', '부채비율_2006', '총자산회전율_2006',
       '매출채권회전율_2006', '재고자산회전율_2006', '순이익율_2009', 'ROE_2009', '총자산증가율_2009',
       '매출액증가율_2009', '유동자산증가율_2009', '유동비율_2009', '부채비율_2009', '총자산회전율_2009',
       '매출채권회전율_2009', '재고자산회전율_2009', '순이익율_2012', 'ROE_2012', '총자산증가율_2012',
       '매출액증가율_2012', '유동자산증가율_2012', '유동비율_2012', '부채비율_2012', '총자산회전율_2012',
       '매출채권회전율_2012', '재고자산회전율_2012'],
      dtype='object')

In [58]:
finance_data.shape

(2321, 63)

In [59]:
#### 21년 1분기 재무정보를 사용하여 재무지수를 만들예정입니다. 필요한 열만 선택해줍니다.
finance_df = finance_data[['종목코드', '종목명', '업종명', 
              '순이익율_2103', 'ROE_2103', 
             '총자산증가율_2103', '매출액증가율_2103', '유동자산증가율_2103', 
             '유동비율_2103', '부채비율_2103',
             '총자산회전율_2103', '매출채권회전율_2103', '재고자산회전율_2103']] 

In [60]:
finance_df = pd.merge(finance_df, marcket_dat, how = 'left', on= '종목코드') #### 시장구분코드를 종목코드를 기중으로 합쳐줍니다.
finance_df.shape

(2321, 14)

In [61]:
#### 컬럼별 결측치 확인
finance_df.isnull().sum() #### NaN값이 많은 것을 확인 할 수 있습니다.

종목코드              0
종목명               0
업종명               0
순이익율_2103        93
ROE_2103        124
총자산증가율_2103      77
매출액증가율_2103     116
유동자산증가율_2103    147
유동비율_2103       139
부채비율_2103        65
총자산회전율_2103     182
매출채권회전율_2103    214
재고자산회전율_2103    299
시장구분              0
dtype: int64

#### (2) 결측치 대체

In [62]:
#### 평균으로 NA를 채우는 함수
fill_mean_func = lambda f : f.fillna(f.mean()) 

In [63]:
finance_df_complete = finance_df.groupby(['업종명', '시장구분']).apply(fill_mean_func).reset_index(drop = True) #### 빈값을 업종별 평균으로 대체해줍니다. 

In [64]:
finance_df_complete.isnull().sum() #### 다시한번 결측치 확인합니다. 그래도 결측치가 남아있는 것을 확인할 수 있습니다.

종목코드             0
종목명              0
업종명              0
순이익율_2103        0
ROE_2103         0
총자산증가율_2103      0
매출액증가율_2103      0
유동자산증가율_2103    48
유동비율_2103       48
부채비율_2103        0
총자산회전율_2103     48
매출채권회전율_2103    48
재고자산회전율_2103    48
시장구분             0
dtype: int64

In [65]:
finance_df_complete[finance_df_complete['유동자산증가율_2103'].isnull()] #### 보험과 은행 증권 업종의 경우 자산에 관한 정보가 없는 것을 확인할 수 있습니다.

,종목코드,종목명,업종명,순이익율_2103,ROE_2103,총자산증가율_2103,매출액증가율_2103,유동자산증가율_2103,유동비율_2103,부채비율_2103,총자산회전율_2103,매출채권회전율_2103,재고자산회전율_2103,시장구분
842,005830,DB손해보험,보험,4.01,10.41,0.58,1.10,NaN,NaN,898.57,NaN,NaN,NaN,KOSPI
843,082640,동양생명,보험,5.52,6.38,-0.31,-0.40,NaN,NaN,1140.50,NaN,NaN,NaN,KOSPI
844,000400,롯데손해보험,보험,7.04,-0.00,-1.63,-0.22,NaN,NaN,1565.63,NaN,NaN,NaN,KOSPI
845,000060,메리츠화재,보험,4.49,19.28,0.62,4.87,NaN,NaN,995.32,NaN,NaN,NaN,KOSPI
846,085620,미래에셋생명,보험,0.01,2.48,0.52,-3.07,NaN,NaN,1842.19,NaN,NaN,NaN,KOSPI
847,032830,삼성생명,보험,11.15,5.86,-0.60,14.26,NaN,NaN,710.03,NaN,NaN,NaN,KOSPI
848,000810,삼성화재,보험,7.09,6.96,-0.53,5.39,NaN,NaN,493.44,NaN,NaN,NaN,KOSPI
849,000815,삼성화재우,보험,7.09,6.96,-0.53,5.39,NaN,NaN,493.44,NaN,NaN,NaN,KOSPI
850,003690,코리안리,보험,1.97,6.38,0.03,-11.92,NaN,NaN,419.98,NaN,NaN,NaN,KOSPI
851,088350,한화생명,보험,4.80,3.91,-0.90,1.68,NaN,NaN,1094.78,NaN,NaN,NaN,KOSPI


In [66]:
finance_df_complete = finance_df.groupby(['시장구분']).apply(fill_mean_func).reset_index(drop = True) #### 업종안에 있는 회사가 모두 NaN일 경우 시장 평균으로 대체해줍니다.

In [67]:
finance_df_complete.isnull().sum() #### 다시한번 결측치를 확인합니다. NaN가 없이 모든 결측치가 대체된 것을 확인할 수 있습니다.

종목코드            0
종목명             0
업종명             0
순이익율_2103       0
ROE_2103        0
총자산증가율_2103     0
매출액증가율_2103     0
유동자산증가율_2103    0
유동비율_2103       0
부채비율_2103       0
총자산회전율_2103     0
매출채권회전율_2103    0
재고자산회전율_2103    0
시장구분            0
dtype: int64

#### (3) 데이터 표준화
###### : 업종별로 크기와 분포가 다양하기 때문에 표준화를 통해 분포를 똑같이 맞추어 영향력을 맞추어 줍니다.

In [68]:
##### Standart scaler함수를 만들어줍니다.
def standard_scaler(x): 
    return ((x - np.mean(x)) / np.std(x))

dat = finance_df_complete.groupby(['업종명','시장구분']).transform(standard_scaler) #### 업종별로 표준화를 해줍니다.

In [69]:
dat.head()

,순이익율_2103,ROE_2103,총자산증가율_2103,매출액증가율_2103,유동자산증가율_2103,유동비율_2103,부채비율_2103,총자산회전율_2103,매출채권회전율_2103,재고자산회전율_2103
0,0.193925,0.247786,0.089850,-0.960161,-0.555360,-0.232981,-0.186314,-0.918875,-0.792256,-0.251382
1,0.810386,-1.668242,-0.243296,-0.258271,-0.232208,-0.170111,-0.106590,-2.285214,-1.032423,-1.344659
2,0.167978,0.499050,0.127673,-0.588694,0.086804,-0.234069,0.951213,1.033168,-0.408934,-0.276472
3,0.167816,0.179977,-0.362463,-1.337243,-0.187304,-0.171502,-0.337488,-0.978124,-0.178495,-0.363113
4,0.334661,0.320232,0.231207,-0.113735,-0.250045,-0.439999,0.075471,0.017354,-0.871597,-0.388815


In [70]:
dat.shape

(2321, 10)

In [71]:
df = pd.concat([finance_df_complete.iloc[:,[0, 1, 2, 13]], dat], axis = 1) #### 표준화한 데이터에 종목코드와 업종명, 시장구분을 합쳐줍니다.

#### (4) 재무지수 만들기
###### :  복합지수방법론을 참고하여 재무항목의 관계와 중요도를 고려한 가중치를 선정하였습니다. 이 가중치를 각 재무항목에 곰하여 재무지수를 만듭니다.

In [72]:
df['수익성'] = 0.5 * (df['순이익율_2103']) + 0.5 * (df['ROE_2103']) 
df['성장성'] = 0.5 * (df['총자산증가율_2103']) + 0.3 * (df['매출액증가율_2103']) + 0.2 * (df['유동자산증가율_2103'])
df['안정성'] = 0.6 * (df['유동비율_2103']) + 0.4 * (df['부채비율_2103'])
df['활동성'] = 0.5 * (df['총자산회전율_2103']) + 0.3 * (df['매출채권회전율_2103']) + 0.2 * (df['재고자산회전율_2103'])

In [73]:
dat = df[['종목코드', '종목명', '업종명','시장구분', '수익성', '성장성', '안정성', '활동성']]

In [74]:
dat[dat['종목코드']=='035760']

,종목코드,종목명,업종명,시장구분,수익성,성장성,안정성,활동성
4,035760,CJ ENM,방송서비스,KOSDAQ,0.327446,0.031474,-0.233811,-0.330565


In [75]:
#### 표준화한 데이터를 사용하였기에 분포가 작아 알아보기 힘들다는 문제점이 있습니다. 
#### 지수에 100을 곱하고 100을 더하여 분산을 100, 평균을 100으로 만들어줍니다. 
dat['수익성지수'] = dat['수익성']*100 + 100 
dat['성장성지수'] = dat['성장성']*100 + 100
dat['안정성지수'] = dat['안정성']*100 + 100
dat['활동성지수'] = dat['활동성']*100 + 100
dat = dat[['종목코드', '종목명', '업종명','시장구분', '수익성지수', '성장성지수', '안정성지수', '활동성지수']]

<ipython-input-75-00bbe060b90e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dat['수익성지수'] = dat['수익성']*100 + 100
<ipython-input-75-00bbe060b90e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dat['성장성지수'] = dat['성장성']*100 + 100
<ipython-input-75-00bbe060b90e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/

In [76]:
dat.to_csv("data/재무지수_데이터.csv", index=False, encoding="utf-8-sig")